In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cvxpy as cp
import random
from sklearn.model_selection import train_test_split

In [ ]:
#import csv for training and testing our model
url_allmovie = 'https://raw.githubusercontent.com/JihoonChung/MIE368_project/master/rotten_tomatoes_full_revised.csv'
df_rotten_tomatoes_full = pd.read_csv(url_allmovie)

#import csv with movie festival data (out of sample)
url_fesitval = 'https://raw.githubusercontent.com/JihoonChung/MIE368_project/master/festival_movie_list.csv'
df_festival_movies = pd.read_csv(url_fesitval)


In [ ]:
column_drop = ['Cast 1','Cast 2','Cast 3','director_name','Title','Year','AUDIENCE score','AUDIENCE count']


X = df_rotten_tomatoes_full.drop(columns = column_drop)
X_train, X_val, y_train, y_val = train_test_split(X,  # all X data
                                                    df_rotten_tomatoes_full['AUDIENCE score'],  # All y data
                                                    test_size=0.30,  # Fraction of data in test set
                                                    shuffle=True,  # Randomly splits the data
                                                    random_state = 3  # Sets random seed for reproducability 
                                                    )

X_train.head()

,TOMATOMETER score,TOMATOMETER Count,Runtime,Genre_Action,Genre_Art&Foreign,Genre_Classics,Genre_Comedy,Genre_Documentary,Genre_Drama,Genre_Horror,Genre_Kids&Family,Genre_Mystery,Genre_Romance,Genre_SciFi,Rating_G,Rating_NC17,Rating_NR,Rating_PG,Rating_PG-13,Rating_R,cast1_oscar_nom,cast1_oscars,cast2_oscar_nom,cast2_oscars,cast3_oscar_nom,cast3_oscars,cast1_FB_likes,cast2_FB_likes,cast3_FB_likes,dir_oscar_nom,dir_oscars,director_awards,director_facebook_likes,cast_total_facebook_likes
70,95.0,19,70,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,488.0,367.0,367.0,0.0,0.0,0.0,232.0,0.0
75,73.0,15,71,0,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,367.0,367.0,367.0,0.0,0.0,0.0,232.0,0.0
889,92.0,7,104,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,367.0,367.0,367.0,0.0,0.0,0.0,232.0,0.0
1210,40.0,190,120,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,8000.0,5000.0,367.0,0.0,0.0,6.0,124.0,0.0
720,63.0,104,82,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,338.0,367.0,956.0,0.0,0.0,0.0,232.0,0.0


# optimization

The model with the best predictions is ________________________. First we will train this model then feed in the our of sample movie festival data to get Audience Score predictions

In [ ]:
df_festival_movies

,Title,TOMATOMETER score,TOMATOMETER Count,AUDIENCE score,AUDIENCE count,Year,Cast 1,Cast 2,Cast 3,director_name,Runtime,Genre_Action,Genre_Art&Foreign,Genre_Classics,Genre_Comedy,Genre_Documentary,Genre_Drama,Genre_Horror,Genre_Kids&Family,Genre_Mystery,Genre_Romance,Genre_SciFi,Rating_G,Rating_NC17,Rating_NR,Rating_PG,Rating_PG-13,Rating_R,cast1_oscar_nom,cast1_oscars,cast2_oscar_nom,cast2_oscars,cast3_oscar_nom,cast3_oscars,cast1_FB_likes,cast2_FB_likes,cast3_FB_likes,dir_oscar_nom,dir_oscars,director_awards,director_facebook_likes,cast_total_facebook_likes
0,The Lion King,53,406,88,76333,1994,Matthew Broderick,Jeremy Irons,James Earl Jones,Roger Allers,87,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,2000,367,367,0,0,0,28,6458
1,Aladdin,57,362,94,59043,1992,Scott Weinger,Robin Williams,Linda Larkin,John Musker,90,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,4,1,0,0,631,49000,367,0,0,0,232,52970
2,Jacob's Ladder,5,22,40,132,1990,Tim Robbins,Elizabeth Pea,Danny Aiello,Adrian Lyne,113,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,367,367,388,1,0,9,213,0
3,Driven,62,50,72,122,2001,Sylvester Stallone,Burt Reynolds,Kip Pardue,Renny Harlin,117,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,2,0,1,0,0,0,13000,367,374,0,0,15,212,14780
4,Burn,58,12,41,44,2012,Donald Austin,"Brendan ""Doogie"" Milewski",Dave Parnell,Brenna Sanchez,86,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,367,367,0,0,0,232,2
5,Love Ranch,13,52,28,1235,2010,Helen Mirren,Joe Pesci,Sergio Peris-Mencheta,Taylor Hackford,117,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,1,3,1,0,0,367,367,367,1,0,0,138,3204
6,A Star Is Born,90,501,79,20023,1937,Janet Gaynor,Fredric March,Adolphe Menjou,William Wellman,111,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,2,1,5,2,1,0,367,367,99,2,0,0,232,0
7,Robin Hood,15,164,41,3497,2010,Russell Crowe,Cate Blanchett,Max von Sydow,Ridley Scott,148,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,3,1,7,2,2,0,367,367,367,3,0,118,0,3244


## coefficent

In [ ]:
def G_gi():
  for i in range(df_festival_movies.shape[0]):
    if i == 0:
      G_gi = df_festival_movies.loc[0][11:22]
    else:
      G_gi = pd.concat([G_gi,df_festival_movies.loc[i][11:22]],axis =1)
  return G_gi

def W_ij():
  timeslots = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
  w ={}
  random.seed(0)
  for i in timeslots:
    random_time = []
    #np.random.RandomState(3)
    zero_count = random.randint(0, df_festival_movies.Title.shape[0])
    one_count = df_festival_movies.Title.shape[0] - zero_count
    random_time = [0]*zero_count + [1]*one_count
    random_time = [0]*zero_count + [1]*one_count
    random.shuffle(random_time)
    w[i] = random_time
    
  # Convert ditionary to DataFrame
  W_ij = pd.DataFrame(w, index = np.array(df_festival_movies.Title))
  return W_ij,timeslots

W_ij ,timeslots = W_ij()
W_ij

,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
The Lion King,1,0,1,1,0,1,0,1,1,0,1,1,0,1,1
Aladdin,0,0,0,1,1,1,0,1,0,1,1,0,0,1,1
Jacob's Ladder,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1
Driven,0,1,1,1,0,1,1,0,1,1,1,1,1,1,1
Burn,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1
Love Ranch,0,0,1,1,1,1,0,1,1,0,1,1,0,0,1
A Star Is Born,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1
Robin Hood,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1


In [ ]:
#np.random.seed(0)
#Coefficent
#W_ij ,timeslots = W_ij()
a_i_real =  cp.reshape(df_festival_movies.transpose().loc['TOMATOMETER score'],(1,8))
a_i_predict = [48.36652423467446,
 46.58315909354554,
 35.202074740924886,
 53.575257485427585,
 53.68249777338187,
 28.571845129427334,
 59.435645701083985,
 27.92678352159285]
r_i = cp.reshape(df_festival_movies.Runtime,(1,8))
C = 13
B = 78
G_gi = G_gi()
print(G_gi)
G_gi.loc['Genre_Action']
W_ij
print(a_i_real)
print(a_i_predict)

                   0  1  2  3  4  5  6  7
Genre_Action       1  1  0  1  0  0  0  1
Genre_Art&Foreign  0  0  0  0  0  0  0  0
Genre_Classics     0  0  0  0  0  0  1  1
Genre_Comedy       0  0  0  0  0  0  0  0
Genre_Documentary  0  0  0  0  1  0  0  0
Genre_Drama        1  0  1  1  0  1  1  1
Genre_Horror       0  0  1  0  0  0  0  0
Genre_Kids&Family  1  1  0  0  0  0  0  1
Genre_Mystery      0  0  1  1  0  0  0  0
Genre_Romance      0  0  0  0  0  0  0  1
Genre_SciFi        0  0  0  0  0  0  0  0
reshape([53. 57.  5. 62. 58. 13. 90. 15.], (1, 8))
[48.36652423467446, 46.58315909354554, 35.202074740924886, 53.575257485427585, 53.68249777338187, 28.571845129427334, 59.435645701083985, 27.92678352159285]


## Decision Variable

In [ ]:
#Decision variable 
num_timeslot = W_ij.shape   # number of possible sites to visit
x_ij = cp.Variable(num_timeslot,boolean=True)  # x is a vector of binary variables of size num_locations.

##Objective Function

In [ ]:
#objective function
obj_real = cp.Maximize(cp.sum(a_i_real*x_ij))
obj_pred = cp.Maximize(cp.sum(a_i_predict*x_ij))


In [ ]:
W_ij

,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
The Lion King,1,0,1,1,0,1,0,1,1,0,1,1,0,1,1
Aladdin,0,0,0,1,1,1,0,1,0,1,1,0,0,1,1
Jacob's Ladder,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1
Driven,0,1,1,1,0,1,1,0,1,1,1,1,1,1,1
Burn,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1
Love Ranch,0,0,1,1,1,1,0,1,1,0,1,1,0,0,1
A Star Is Born,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1
Robin Hood,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1


## Constraints

In [ ]:
def genreConst(genre,time):
  for i in G_gi.loc[genre,:]:
    constraint = []
    constraint.append(sum(i*x_ij[i]))
  cons.append(sum(constraint)>=time)

def overlapping(interval):
  for i in range(len(timeslots)):
    constraint = []
    constraint.append(sum(x_ij[:,i]))
    for j in timeslots[i+1:]:
      if timeslots[i]+interval > j:
        constraint.append(sum(x_ij[:,timeslots.index(j)]))
    cons.append(sum(constraint)<=1)

def rewatchTime(numberoftime):
  for i in range(len(W_ij.index)):
    cons.append(sum(x_ij[i,:])<= numberoftime)

#gotta fix
def horror_time(time):
  constraint = []
  for j in range(len(timeslots)):
    if j < time: 
      constraint.append(sum(G_gi.loc['Genre_Horror']*x_ij[:,j]))
  cons.append(sum(constraint)==0)

def Balance(start,finish,maxNum):
  cons.append(sum(x_ij[:,start:finish])<= maxNum)


In [ ]:
cons=[]
# Cost of all movie tickets must be less than or equal to the budget
cons.append(sum(C*x_ij)<=B)

#W_ij
cons.append(x_ij <= W_ij)

#Total time watching movies <= 10 hours
cons.append(cp.sum(r_i*x_ij))

# See one movie max 1 time 
rewatchTime(1)

#Can’t watch overlapping movies. (See picture below)
overlapping(2.5)

#At least one Action movie (or whatever genre) 
genreConst('Genre_Action',1)

#Balance schedule t
Balance(0,3,2)
Balance(3,15,3)

In [ ]:
#The audience don’t want to watch a horror movie before 6 pm
horror_time(6)

## Solve

In [ ]:
prob = cp.Problem(obj_real,cons)
prob.solve(verbose=False)

print(x_ij.value)

prob = cp.Problem(obj_pred,cons)
prob.solve(verbose=False)
print("")
print(x_ij.value)


[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


'\nprob = cp.Problem(obj_pred,cons)\nprob.solve(verbose=False)\nprint("")\nprint(x_ij.value)\n'

## results

In [ ]:
df_schedule = pd.DataFrame(W_ij)
df_schedule


,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
The Lion King,1,0,1,1,0,1,0,1,1,0,1,1,0,1,1
Aladdin,0,0,0,1,1,1,0,1,0,1,1,0,0,1,1
Jacob's Ladder,0,0,1,1,1,1,0,1,1,0,1,1,1,1,1
Driven,0,1,1,1,0,1,1,0,1,1,1,1,1,1,1
Burn,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1
Love Ranch,0,0,1,1,1,1,0,1,1,0,1,1,0,0,1
A Star Is Born,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1
Robin Hood,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1


In [ ]:
df_iternary = pd.DataFrame(x_ij.value.astype(int))#,column = timeslots)#,index = df_festival_movies.Title)

df_iternary.columns = timeslots
df_iternary.index = df_festival_movies.Title

df_iternary

,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Title,,,,,,,,,,,,,,,
The Lion King,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Aladdin,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Jacob's Ladder,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Driven,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Burn,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Love Ranch,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A Star Is Born,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Robin Hood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print('The maximum real utility(audience score) is {:.3f}'.format(obj_real.value))
print('The maximum predict utility(audience score) is {:.3f}'.format(obj_pred.value))


The maximum real utility(audience score) is 320.000
The maximum predict utility(audience score) is 261.643


In [ ]:
df_iternary_copy = df_iternary.replace(0,np.nan)
df_iternary_copy = df_iternary_copy.replace(1,np.nan)
df_iternary_copy

,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Title,,,,,,,,,,,,,,,
The Lion King,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aladdin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jacob's Ladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Driven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Love Ranch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Star Is Born,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Robin Hood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
